### This notebook first labels clusters of protease inhibitors from ProteinCartography run
Full outputs from the final cartography run (carto_run3) on Zenodo:  tick_PUFs_PIs_1200_plus_toxinDB_carto_run3.zip at https://zenodo.org/records/15186244


#### Input:
Umap clustering (copied over from full PC run): ../outputs/carto_run/carto_output_run3/clusteringresults/carto_run3_aggregated_features_pca_umap.tsv \
Leiden similarity (copied over from full PC run): ../outputs/carto_run/carto_output_run3/clusteringresults/carto_run3_leiden_similarity.tsv \
Structural similarity (copied over from full PC run): ../outputs/carto_output_run3/carto_run/clusteringresults/carto_run3_strucluster_similarity.tsv 


#### Output:
Upated umap with protease inhibitor cluster names added: ../datasheets/carto_run3_aggregated_features_pca_umap_updated.tsv \
High confidence protease inhibitor orthogroups: ../datasheets/PI_orthogroups_high_qual_06112023.tsv \
Low confidence protease inhibitor orthogroups: ../datasheets/PI_orthogroups_low_qual_06112023.tsv 


Ultimately, I found 7 clusters that are high quality protease inhibitors (2.5k proteins) and 4 clusters that are interesting but are much lower confidence (1.8 k proteins).  

### Then, this notebook uses the structural clusters to find orthogroups of protease inhibitors to move forward for trait mapping

#### Input:

Orthogroup data: Sourced from [Comparative phylogenomic analysis of chelicerates points to gene families associated with long-term suppression of host detection](https://dx.doi.org/10.57844/arcadia-4e3b-bbea)\
Same copy of orthogroup data can also be found here: ../datasheets/Orthogroups.tsv

#### Output:

Orthogroups with high confidence (HQ) PI calls: ../datasheets/PI_orthogroups_high_qual_06112023.tsv \
Orthogroups with low confidence (LQ) PI calls: ../datasheets/PI_orthogroups_low_qual_06112023.tsv

This analysis results in 14 orthogroups of high-confidence protease inhibitors, and 22 orthogroups of low confidence protease inhibitors. These 36 orthogroups were moved forward to phylogenetic profiling to associate them with the host detection suppression trait.


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np
import glob
from collections import defaultdict
pd.set_option('display.max_columns', None)

## Reading in umap clustering of structures

In [2]:
#umap clustering from cartography
umap = pd.read_csv("../outputs/carto_run/carto_output_run3/clusteringresults/carto_run3_aggregated_features_pca_umap.tsv", sep = "\t")
umap = umap.loc[umap["Sequence"].notna()]


/var/folders/rb/9psz38xj6h7685fqhq0kwtc40000gn/T/ipykernel_31380/175374996.py:2: DtypeWarning: Columns (9,10,12,15,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  umap = pd.read_csv("../outputs/carto_run/carto_output_run3/clusteringresults/carto_run3_aggregated_features_pca_umap.tsv", sep = "\t")


### Reading in cluster self-similarity TM scores to see how well the structures were clustered in this analysis
0.2 TM is the minimum TM score that could be considered a non-random match, so we will be highly suspicious of clusters below this threshold. Also should treat clusters with 0.2-0.5 with scrutiny, above 0.5 is generally recgonized to be the same whereas lower is less certain. ProteinCartography provided two different analyses of cluster similarity - Leiden similarity (self similarity of Leiden Clusters made in this analysis) as well as Structural Cluster similarity (Foldseek default clustering). I'm going to read in both metrics, but in downstream analyses I end up using Leiden clusters , as there are far fewer of them than the Foldseek "Structural clusters" and they appear to represent the different major classes of proteins well.  

In [3]:
#read in self similarity - 0.2 TM is the minimum TM score that could be considered a non-random match, so we will want to remove clusters with a self similarity below this
#also shoudl treat clusters with 0.2-0.5 with scrutiny, above 0.5 is generally recgonized to be the same wherease lower is sketchier 
LC_self_similarity = pd.read_csv("../outputs/carto_run/carto_output_run3/clusteringresults/carto_run3_leiden_similarity.tsv", sep = "\t",index_col=0)
ST_self_similarity = pd.read_csv("../outputs/carto_run/carto_output_run3/clusteringresults/carto_run3_strucluster_similarity.tsv", sep = "\t",index_col=0)


In [5]:
#add self sim information to the big umap dataframe 
sim_dict = defaultdict(list)
for cluster in ST_self_similarity.index.values.tolist():
    TM = ST_self_similarity.loc[[cluster],[cluster]].values[0][0]
    sim_dict["StruCluster"].append(cluster)
    sim_dict["within_ST_TM"].append(TM)
    
ST_sim_df = pd.DataFrame(sim_dict)
umap = umap.merge(ST_sim_df, on = "StruCluster", how = "left")


#### Next, I explored ProteinCartography visualization and semantic analyses of the Leiden Clusters. Doing this, I identified 7 high confidence protease inhibitor clusters. High confidence means that they have a within-cluster TM score >0.2 and annotations consistent with protease inhibition function. 

- LC06: Kunitz domain protease inhibitors
- LC13: Kunitz domain protease inhibitors
- LC14: Serpin domain protease inhibitors
- LC15: Cysteine-rich secretory proteins (CRISPs), related to some protease inhibitors
- LC20: Serpin domain protease inhibitors
- LC22: Cystatin protease inhibitors
- LC29: Trypsin-inhibitor-like (TIL) protease inhibitors



In [6]:
#adding shorthand names to the dataframe so I can keep track of which cluster is which
name_dict = {"LC06": 'Kunitz-2',
             "LC13": 'Kunitz-1',
             "LC14": 'Serpin-1',
             "LC15": 'CRISP-PIs',
             "LC20": 'Serpin-2',
             "LC22": 'Cystatin',
             "LC29": "TIL"}
umap["PI_cluster"] = umap['LeidenCluster'].map(name_dict)

#### Beyone the 7 "high confidence" protease inhibitor clusters, I also identified 3 "lower confidence" clusters. These clusters either had low within-cluster TM scores or weak annotations

- LC02: TM score < 0.2, Kazal and insulin growth factor (IGF) annotations
- LC05: TM score < 0.2, Kazal and Alpha-2 macroglobulin (A2M) annotations
- LC17: TM score > 0.2, weak annotations related to Kazal domains
- LC21: TM score < 0.2, Kazal and Whey Acidic Protein (WAP) annotations


In [7]:
#dictionary with sketchy clusters and shorthand names
sketch_dict = {"LC02": 'BadTM-Kazal-IGF',
             "LC05": 'BadTM-Kazal-A2M',
             "LC17": 'WeakAnnotation-Kazal',
             "LC21": 'BadTM-Kazal-WAP'}

umap["Low_Quality_PI_cluster"] = umap['LeidenCluster'].map(sketch_dict)

In [8]:
#adding this cluster name info to the umap csv, putting in datasheets folder
umap.to_csv("../datasheets/carto_run3_aggregated_features_pca_umap_updated.tsv", sep = "\t", index = False)

### The next step is to use our NovelTree analysis to identify the gene families (orthogroups) that underlie these structural clusters. 
This will enable us perform trait mapping to see which ones are postively associated with host detection-supression trait

### Reading in full set of chelicerate proteins that went into NovelTree 

In [9]:
#reading in all annoations for the full chelicerate protein dataset (novel tree input) 
all_proteins = pd.read_csv("../../25aacutoff_fullset_100223_annotated/all_chelicerate_proteins_annotated.csv")
# reformatting Organism to match with NovelTree
all_proteins["Organism"] = all_proteins["species_name"].apply(lambda x: x.replace("-"," "))

### Reading in and reshaping orthogroup table from NovelTree
To intersect with my analyses,  I needed to reshape the orthogroups dataframe the dataframe has 1 column with gene names and the other column with orthogroup number 

In [10]:
#from NovelTree run - see https://research.arcadiascience.com/pub/result-chelicerate-detection-suppression/release/2
orthogroups = pd.read_csv("../datasheets/Orthogroups.tsv", sep = "\t")

In [11]:
# Use melt to reshape the DataFrame
melted_df = orthogroups.melt(id_vars='Orthogroup', var_name='Organism', value_name='gene_name')

# Drop the 'Organism' column
reshaped_df = melted_df.drop(columns='Organism')
reshaped_df = reshaped_df.dropna()
# Create a copy of reshaped_df
expanded_df = reshaped_df.copy()

# Split the 'Gene Name' column on commas
expanded_df['gene_name'] = expanded_df['gene_name'].str.split(', ')

# Use explode to create a new row for each gene name
expanded_df = expanded_df.explode('gene_name')
expanded_df.reset_index( drop = True, inplace=True)

### Merging orthogroup information with full list of Chelicerate proteins

In [12]:
all_proteins = all_proteins.merge(expanded_df, on = "gene_name", how = 'outer', indicator = True)


### Adding in the structural cluster information  

In [13]:
#just grabbing info that is unique to this df so i can intersect it back with the larger chelicerate dataset 
umap_small = umap[["protid", "PI_or_PUF","StruCluster", "LeidenCluster", "within_LC_TM", "PI_cluster", "Low_Quality_PI_cluster"]]
#adding in umap information from cartography 
all_proteins_merged = all_proteins.merge(umap_small, left_on = "gene_name", right_on = "protid", how = "left")

In [14]:
#writing out large file of all chelicerate proteins (too big for github)
all_proteins.to_csv("../../25aacutoff_fullset_100223_annotated/all_chelicerate_proteins_annotated_plus_orthogroups_plus_clusters.csv", index = False)

In [15]:
#subsetting down to tick proteins (only have structural cluster information for tick proteins anway)
tick_proteins = all_proteins_merged.loc[all_proteins_merged["is_tick"] =="yes"]

### Getting counts and stats for different orthogroups are in each structural cluster to pick the orthogroups to move forward for trait mapping

In [16]:
#HQ = high confidence PI clusters 

HQ_target_PIs = tick_proteins.loc[tick_proteins["PI_cluster"].notna()]

#LQ = low confidence PI clustes (sketchy clusters)
LQ_target_PIs = tick_proteins.loc[tick_proteins["Low_Quality_PI_cluster"].notna()]

In [17]:
def get_ortho_counts(ortho_list, name_col):
    #take a list of orthogroups and a column of their names (like PI cluster names) and count up how many proteins are in the OG
    #also counts how many proteins are secreted, how many proteins were in carto, how many tick speices are in OG 
    
    num_dict = defaultdict(list)

    for orthogroup in ortho_list:
        # proteins that were part of cartography run 
        carto = tick_proteins.loc[(tick_proteins["Orthogroup"] == orthogroup) & (tick_proteins["LeidenCluster"].notna())]
        
        # secreted carto proteins 
        carto_secreted = tick_proteins.loc[(tick_proteins["Orthogroup"] == orthogroup) & (tick_proteins["deepsig_feature"] == "Signal peptide") & (tick_proteins["LeidenCluster"].notna())]

        # all tick proteins
        all_tick = tick_proteins.loc[(tick_proteins["Orthogroup"] == orthogroup) ]

        #secreted tick proteins
        tick_secreted = tick_proteins.loc[(tick_proteins["Orthogroup"] == orthogroup) & (tick_proteins["deepsig_feature"] == "Signal peptide")]
        #names associated with orthogroup 
        PI_names = set(tick_proteins.loc[tick_proteins["Orthogroup"] == orthogroup][name_col])
    
        num_dict["Orthogroup"].append(orthogroup)
        num_dict["species_total"].append(len(set(all_tick["Organism"])))
        num_dict["carto_total"].append(len(carto))
        num_dict["carto_secreted"].append(len(carto_secreted))
        num_dict["tick_total"].append(len(all_tick))
        num_dict["tick_secreted"].append(len(tick_secreted))
        num_dict["PI_cluster"].append(PI_names)

        #print(orthogroup, name_col, PI_names)
        
        

    return(num_dict)
    

### Getting orthogroups and their sizes for the good high quality (HQ)  structural clusters 

In [18]:
HQ_PI_ortho = set(HQ_target_PIs.loc[HQ_target_PIs["Orthogroup"].notna()]["Orthogroup"])
HQ_ortho_counts = get_ortho_counts(HQ_PI_ortho,"PI_cluster" )

### Removing small orthogroups and/or orthogroups with a low number of secreted proteins
This gives us 14 orthogroups of high confidence protease inhibitors

In [19]:
HQ_num_df = pd.DataFrame(HQ_ortho_counts)

#adding some more calculations for filtering
HQ_num_df["percent_in_carto"] = 100*HQ_num_df["carto_total"]/HQ_num_df["tick_total"]
HQ_num_df["percent_carto_secreted"] = 100*HQ_num_df["carto_secreted"]/HQ_num_df["carto_total"]
HQ_num_df["percent_tick_secreted"] = 100*HQ_num_df["tick_secreted"]/HQ_num_df["tick_total"]

#set some filtering criteria to get rid of really small orthogroups, as well as orthogroups without a lot of secreted proteins 

HQ_num_df_filtered = HQ_num_df.loc[HQ_num_df["tick_secreted"] >=5]
HQ_num_df_filtered = HQ_num_df_filtered.loc[HQ_num_df_filtered["percent_tick_secreted"] >=5]
HQ_num_df_filtered = HQ_num_df_filtered.loc[HQ_num_df_filtered["species_total"]>=10]



### Getting orthogroups and their sizes for the low quality (LQ)  structural clusters 

In [20]:
LQ_PI_ortho = set(LQ_target_PIs.loc[LQ_target_PIs["Orthogroup"].notna()]["Orthogroup"])
LQ_ortho_counts = get_ortho_counts(LQ_PI_ortho,"Low_Quality_PI_cluster" )

### Removing small orthogroups and/or orthogroups with a low number of secreted proteins. 
This gives us 22 orthogroups of low confidence protease inhibitors

In [21]:
LQ_num_df = pd.DataFrame(LQ_ortho_counts)
LQ_num_df["percent_in_carto"] = 100*LQ_num_df["carto_total"]/LQ_num_df["tick_total"]
LQ_num_df["percent_carto_secreted"] = 100*LQ_num_df["carto_secreted"]/LQ_num_df["carto_total"]
LQ_num_df["percent_tick_secreted"] = 100*LQ_num_df["tick_secreted"]/LQ_num_df["tick_total"]

#set some filtering criteria to get rid of really small orthogroups, as well as orthogroups without a lot of secreted proteins 
#i used more stringent filtering criteria for these sketchy clusters compared to the good clusters 

LQ_num_df_filtered = LQ_num_df.loc[LQ_num_df["tick_secreted"] >=10]
LQ_num_df_filtered = LQ_num_df_filtered.loc[LQ_num_df_filtered["percent_tick_secreted"] >=25]
LQ_num_df_filtered = LQ_num_df_filtered.loc[LQ_num_df_filtered["percent_in_carto"] >=50]

LQ_num_df_filtered = LQ_num_df_filtered.loc[LQ_num_df_filtered["species_total"] >=10]



### Writing out two seperate files for low and high confidence protease inhibitor orthogroups

In [22]:
len(HQ_num_df_filtered)

14

In [23]:
HQ_num_df_filtered

,Orthogroup,species_total,carto_total,carto_secreted,tick_total,tick_secreted,PI_cluster,percent_in_carto,percent_carto_secreted,percent_tick_secreted
9,OG0000039,15,408,202,424,204,"{Kunitz-1, nan, Kunitz-2}",96.226415,49.509804,48.113208
23,OG0000230,15,79,26,86,27,"{CRISP-PIs, nan}",91.860465,32.911392,31.395349
28,OG0008132,12,19,12,20,12,"{Kunitz-1, nan, Kunitz-2}",95.000000,63.157895,60.000000
31,OG0002510,12,2,2,42,12,"{TIL, nan}",4.761905,100.000000,28.571429
41,OG0005278,10,36,33,39,33,"{Cystatin, nan}",92.307692,91.666667,84.615385
43,OG0006419,13,11,6,13,6,"{Cystatin, nan}",84.615385,54.545455,46.153846
74,OG0000017,15,476,164,478,164,"{Serpin-2, Serpin-1, nan}",99.581590,34.453782,34.309623
93,OG0000058,15,119,118,313,204,"{TIL, nan}",38.019169,99.159664,65.175719
97,OG0001121,14,21,5,22,5,"{nan, Kunitz-2}",95.454545,23.809524,22.727273
118,OG0000216,15,135,98,137,98,"{Cystatin, nan}",98.540146,72.592593,71.532847


In [24]:
len(LQ_num_df_filtered)

22

In [25]:
LQ_num_df_filtered

,Orthogroup,species_total,carto_total,carto_secreted,tick_total,tick_secreted,PI_cluster,percent_in_carto,percent_carto_secreted,percent_tick_secreted
7,OG0000677,13,50,50,72,56,"{BadTM-Kazal-IGF, nan}",69.444444,100.000000,77.777778
41,OG0007096,13,11,11,14,11,"{BadTM-Kazal-IGF, nan}",78.571429,100.000000,78.571429
51,OG0009247,11,11,11,13,11,"{BadTM-Kazal-A2M, nan}",84.615385,100.000000,84.615385
52,OG0009059,14,12,12,15,12,"{WeakAnnotation-Kazal, nan}",80.000000,100.000000,80.000000
71,OG0003200,14,37,37,48,37,"{BadTM-Kazal-IGF, nan}",77.083333,100.000000,77.083333
105,OG0005859,15,16,16,30,16,"{WeakAnnotation-Kazal, nan}",53.333333,100.000000,53.333333
113,OG0006622,12,11,11,13,11,"{WeakAnnotation-Kazal, nan}",84.615385,100.000000,84.615385
163,OG0000480,14,33,5,52,18,"{BadTM-Kazal-A2M, nan}",63.461538,15.151515,34.615385
164,OG0001145,13,20,20,36,20,"{WeakAnnotation-Kazal, nan}",55.555556,100.000000,55.555556
199,OG0001324,15,55,55,70,55,"{BadTM-Kazal-IGF, nan}",78.571429,100.000000,78.571429


In [56]:
#writing out target orthogroups - handing off for trait mapping analysis 
HQ_num_df_filtered.to_csv("../datasheets/PI_orthogroups_high_qual_06112023.tsv", sep = "\t", index = False)
LQ_num_df_filtered.to_csv("../datasheets/PI_orthogroups_low_qual_06112023.tsv", sep = "\t", index = False)